In [1]:
import boto3

In [2]:
rsd_client = boto3.client('redshift-data')

In [3]:
query_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_dm',
    Sql='SELECT current_date'
)['Id']

In [4]:
rsd_client.describe_statement(Id=query_id)

{'CreatedAt': datetime.datetime(2022, 7, 21, 17, 55, 45, 895000, tzinfo=tzlocal()),
 'Duration': 6128724,
 'HasResultSet': True,
 'Id': '0224d62b-e3bc-48e9-a60a-7ae77cefc52f',
 'QueryString': 'SELECT current_date',
 'RedshiftPid': 1073930947,
 'RedshiftQueryId': 0,
 'ResultRows': 1,
 'ResultSize': 10,
 'Status': 'FINISHED',
 'UpdatedAt': datetime.datetime(2022, 7, 21, 17, 55, 46, 583000, tzinfo=tzlocal()),
 'WorkgroupName': 'retail-wg',
 'ResponseMetadata': {'RequestId': '59929a2e-0969-4b02-8832-97df966ed21e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '59929a2e-0969-4b02-8832-97df966ed21e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '302',
   'date': 'Thu, 21 Jul 2022 12:25:58 GMT'},
  'RetryAttempts': 0}}

In [7]:
from botocore.exceptions import WaiterError
from botocore.waiter import WaiterModel
from botocore.waiter import create_waiter_with_client

In [8]:
waiter_name = 'DataAPIExecution'

delay=2
max_attempts=3

#Configure the waiter settings
waiter_config = {
  'version': 2,
  'waiters': {
    'DataAPIExecution': {
      'operation': 'DescribeStatement',
      'delay': delay,
      'maxAttempts': max_attempts,
      'acceptors': [
        {
          "matcher": "path",
          "expected": "FINISHED",
          "argument": "Status",
          "state": "success"
        },
        {
          "matcher": "pathAny",
          "expected": ["PICKED","STARTED","SUBMITTED"],
          "argument": "Status",
          "state": "retry"
        },
        {
          "matcher": "pathAny",
          "expected": ["FAILED","ABORTED"],
          "argument": "Status",
          "state": "failure"
        }
      ],
    },
  },
}

In [9]:
waiter_model = WaiterModel(waiter_config)

In [10]:
custom_waiter = create_waiter_with_client(
    waiter_name, waiter_model, rsd_client
)

In [11]:
query_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_dm',
    Sql='SELECT current_date'
)['Id']

try:
    custom_waiter.wait(Id=query_id)    
except WaiterError as e:
    print (e)


In [12]:
rsd_client.describe_statement(Id=query_id)

{'CreatedAt': datetime.datetime(2022, 7, 21, 18, 12, 52, 30000, tzinfo=tzlocal()),
 'Duration': 5536075,
 'HasResultSet': True,
 'Id': '9941257d-1933-4d4d-b752-6028934bda1c',
 'QueryString': 'SELECT current_date',
 'RedshiftPid': 1073988292,
 'RedshiftQueryId': 0,
 'ResultRows': 1,
 'ResultSize': 10,
 'Status': 'FINISHED',
 'UpdatedAt': datetime.datetime(2022, 7, 21, 18, 12, 52, 815000, tzinfo=tzlocal()),
 'WorkgroupName': 'retail-wg',
 'ResponseMetadata': {'RequestId': 'c78cb6c5-80eb-405e-8c5c-d16eef095ecc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c78cb6c5-80eb-405e-8c5c-d16eef095ecc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '301',
   'date': 'Thu, 21 Jul 2022 12:43:15 GMT'},
  'RetryAttempts': 0}}